In [45]:
# primeira tentativa com random forest

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
import joblib

# Carregar os dados
df = pd.read_csv("Data/Dataframelimpa_sem_latlong.csv")

# Pré-processamento dos Dados
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# Função para extrair características de data/hora
def extract_datetime_features(df):
    df['purchase_weekday'] = df['order_purchase_timestamp'].dt.weekday
    df['purchase_month'] = df['order_purchase_timestamp'].dt.month
    df['purchase_hour'] = df['order_purchase_timestamp'].dt.hour
    df['approval_delay'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # em horas
    return df

df = extract_datetime_features(df)

# Variáveis preditoras e alvo
X = df[['purchase_weekday', 'purchase_month', 'purchase_hour', 'approval_delay']]
y = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # Prever em horas

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline de pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['approval_delay']),
        ('cat', OneHotEncoder(), ['purchase_weekday', 'purchase_month', 'purchase_hour'])
    ])

# Pipeline completo
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Treinar o modelo
model_pipeline.fit(X_train, y_train)

# Fazer previsões
y_pred = model_pipeline.predict(X_test)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse}')

# Salvar o modelo para uso futuro
joblib.dump(model_pipeline, 'modelo_previsao_data_entrega.pkl')

# Exemplo de uso do modelo salvo em novos dados
modelo_carregado = joblib.load('modelo_previsao_data_entrega.pkl')


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
from concurrent.futures import ThreadPoolExecutor

# Carregar os dados
df = pd.read_csv("Data/Dataframelimpa_sem_latlong.csv")

# Pré-processamento dos Dados
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# Função para extrair características de data/hora
def extract_datetime_features(df):
    df['purchase_weekday'] = df['order_purchase_timestamp'].dt.weekday
    df['purchase_month'] = df['order_purchase_timestamp'].dt.month
    df['purchase_hour'] = df['order_purchase_timestamp'].dt.hour
    df['approval_delay'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # em horas
    return df

# Calcular o volume do produto
df['product_volume'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']

df = extract_datetime_features(df)

# Garantir que não haja valores NaN nos dados
df = df.fillna(0)

# Variáveis preditoras e alvo
X = df[['purchase_weekday', 'purchase_month', 'purchase_hour', 'approval_delay', 'product_volume']]
y = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # Prever em horas

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline de pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['approval_delay', 'product_volume']),
        ('cat', OneHotEncoder(), ['purchase_weekday', 'purchase_month', 'purchase_hour'])
    ])

# Pipeline completo
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
])

# Usar ThreadPoolExecutor para treino do modelo
def train_model(pipeline, X_train, y_train):
    pipeline.fit(X_train, y_train)

with ThreadPoolExecutor() as executor:
    future = executor.submit(train_model, model_pipeline, X_train, y_train)
    future.result()  # Esperar a conclusão do treino

# Fazer previsões
y_pred = model_pipeline.predict(X_test)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'MSE: {mse}')
print(f'MAE: {mae}')

# Salvar o modelo para uso futuro
joblib.dump(model_pipeline, 'modelo_previsao_data_entrega.pkl')

# Exemplo de uso do modelo salvo em novos dados
modelo_carregado = joblib.load('modelo_previsao_data_entrega.pkl')


MSE: 18521.300670029792
MAE: 105.63585213014834


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
from concurrent.futures import ThreadPoolExecutor

# Carregar os dados
df = pd.read_csv("Data/Dataframelimpa_sem_latlong.csv")

# Pré-processamento dos Dados
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# Função para extrair características de data/hora
def extract_datetime_features(df):
    df['purchase_weekday'] = df['order_purchase_timestamp'].dt.weekday
    df['purchase_month'] = df['order_purchase_timestamp'].dt.month
    df['purchase_hour'] = df['order_purchase_timestamp'].dt.hour
    df['approval_delay'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # em horas
    return df

# Calcular o volume do produto
df['product_volume'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']

df = extract_datetime_features(df)

# Garantir que não haja valores NaN nos dados
df = df.fillna(0)

# Variáveis preditoras e alvo
X = df[['purchase_weekday', 'purchase_month', 'purchase_hour', 'approval_delay', 'product_volume', 'product_weight_g']]
y = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # Prever em horas

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline de pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['approval_delay', 'product_volume', 'product_weight_g']),
        ('cat', OneHotEncoder(), ['purchase_weekday', 'purchase_month', 'purchase_hour'])
    ])

# Pipeline completo
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
])

# Usar ThreadPoolExecutor para treino do modelo
def train_model(pipeline, X_train, y_train):
    pipeline.fit(X_train, y_train)

with ThreadPoolExecutor() as executor:
    future = executor.submit(train_model, model_pipeline, X_train, y_train)
    future.result()  # Esperar a conclusão do treino

# Fazer previsões
y_pred = model_pipeline.predict(X_test)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'MSE: {mse}')
print(f'MAE: {mae}')

# Salvar o modelo para uso futuro
joblib.dump(model_pipeline, 'modelo_previsao_data_entrega.pkl')

# Exemplo de uso do modelo salvo em novos dados
modelo_carregado = joblib.load('modelo_previsao_data_entrega.pkl')


MSE: 18136.803027188955
MAE: 104.82506590254518


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
from concurrent.futures import ThreadPoolExecutor

# Carregar os dados
df = pd.read_csv("Data/Dataframelimpa_sem_latlong.csv")

# Pré-processamento dos Dados
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# Função para extrair características de data/hora
def extract_datetime_features(df):
    df['purchase_weekday'] = df['order_purchase_timestamp'].dt.weekday
    df['purchase_month'] = df['order_purchase_timestamp'].dt.month
    df['purchase_hour'] = df['order_purchase_timestamp'].dt.hour
    df['approval_delay'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # em horas
    return df

# Função para calcular a similaridade dos prefixos de CEP
def calculate_zip_similarity(df):
    def compare_zip_codes(row):
        seller_zip = str(row['seller_zip_code_prefix'])
        customer_zip = str(row['customer_zip_code_prefix'])
        similarity = sum(1 for s, c in zip(seller_zip, customer_zip) if s == c)
        return similarity

    df['zip_code_similarity'] = df.apply(compare_zip_codes, axis=1)
    return df

# Calcular o volume do produto
df['product_volume'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']

df = extract_datetime_features(df)
df = calculate_zip_similarity(df)

# Garantir que não haja valores NaN nos dados
df = df.fillna(0)

# Variáveis preditoras e alvo
X = df[['purchase_weekday', 'purchase_month', 'purchase_hour', 'approval_delay', 'product_volume', 'product_weight_g', 'zip_code_similarity']]
y = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # Prever em horas

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline de pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['approval_delay', 'product_volume', 'product_weight_g', 'zip_code_similarity']),
        ('cat', OneHotEncoder(), ['purchase_weekday', 'purchase_month', 'purchase_hour'])
    ])

# Pipeline completo
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
])

# Usar ThreadPoolExecutor para treino do modelo
def train_model(pipeline, X_train, y_train):
    pipeline.fit(X_train, y_train)

with ThreadPoolExecutor() as executor:
    future = executor.submit(train_model, model_pipeline, X_train, y_train)
    future.result()  # Esperar a conclusão do treino

# Fazer previsões
y_pred = model_pipeline.predict(X_test)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'MSE: {mse}')
print(f'MAE: {mae}')

# Salvar o modelo para uso futuro
joblib.dump(model_pipeline, 'modelo_previsao_data_entrega.pkl')

# Exemplo de uso do modelo salvo em novos dados
modelo_carregado = joblib.load('modelo_previsao_data_entrega.pkl')


MSE: 18007.406524008937
MAE: 104.4635009544379


In [1]:
# o melhor até agora, pois usou o freight_value, mas ta prevendo a data e nao o delivery time

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
from concurrent.futures import ThreadPoolExecutor

# Carregar os dados
df = pd.read_csv("Data/DataFramelimpa_sem_latlong.csv")

# Pré-processamento dos Dados
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# Função para extrair características de data/hora
def extract_datetime_features(df):
    df['purchase_weekday'] = df['order_purchase_timestamp'].dt.weekday
    df['purchase_month'] = df['order_purchase_timestamp'].dt.month
    df['purchase_hour'] = df['order_purchase_timestamp'].dt.hour
    df['approval_delay'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # em horas
    return df

# Função para calcular a similaridade dos prefixos de CEP
def calculate_zip_similarity(df):
    def compare_zip_codes(row):
        seller_zip = str(row['seller_zip_code_prefix'])
        customer_zip = str(row['customer_zip_code_prefix'])
        similarity = sum(1 for s, c in zip(seller_zip, customer_zip) if s == c)
        return similarity

    df['zip_code_similarity'] = df.apply(compare_zip_codes, axis=1)
    return df

# Calcular o volume do produto
df['product_volume'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']

# Adicionar a variável de frete (freight_value)
df['freight_value'] = df['freight_value']

df = extract_datetime_features(df)
df = calculate_zip_similarity(df)

# Garantir que não haja valores NaN nos dados
df = df.fillna(0)

# Variáveis preditoras e alvo
X = df[['purchase_weekday', 'purchase_month', 'purchase_hour', 'approval_delay', 'product_volume', 'product_weight_g', 'zip_code_similarity', 'freight_value', 'price']]
y = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.total_seconds() / 86400  # Prever em horas

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline de pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['approval_delay', 'product_volume', 'product_weight_g', 'zip_code_similarity', 'freight_value', 'price']),
        ('cat', OneHotEncoder(), ['purchase_weekday', 'purchase_month', 'purchase_hour'])
    ])

# Pipeline completo
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
])

# Usar ThreadPoolExecutor para treino do modelo
def train_model(pipeline, X_train, y_train):
    pipeline.fit(X_train, y_train)

with ThreadPoolExecutor() as executor:
    future = executor.submit(train_model, model_pipeline, X_train, y_train)
    future.result()  # Esperar a conclusão do treino

# Fazer previsões
y_pred = model_pipeline.predict(X_test)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'MSE: {mse}')
print(f'MAE: {mae}')

# Salvar o modelo para uso futuro
joblib.dump(model_pipeline, 'modelo_previsao_data_entrega.pkl')

# Exemplo de uso do modelo salvo em novos dados
modelo_carregado = joblib.load('modelo_previsao_data_entrega.pkl')

MSE: 18.415524473382906
MAE: 3.155886588519814


In [15]:
# melhor até agora, retorna direto o delivery time
# retorna um csv resultado_previsoes que compara o nosso modelo com o antigo fornecido na AWS

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
import joblib
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime, timedelta

# Carregar os dados
df = pd.read_csv("Data/DataFramelimpa_sem_latlong.csv")

# Pré-processamento dos Dados
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# Função para extrair características de data/hora
def extract_datetime_features(df):
    df['purchase_weekday'] = df['order_purchase_timestamp'].dt.weekday
    df['purchase_month'] = df['order_purchase_timestamp'].dt.month
    df['purchase_hour'] = df['order_purchase_timestamp'].dt.hour
    df['approval_delay'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # em horas
    return df

# Função para calcular a similaridade dos prefixos de CEP
def calculate_zip_similarity(df):
    def compare_zip_codes(row):
        seller_zip = str(row['seller_zip_code_prefix'])
        customer_zip = str(row['customer_zip_code_prefix'])
        similarity = sum(1 for s, c in zip(seller_zip, customer_zip) if s == c)
        return similarity

    df['zip_code_similarity'] = df.apply(compare_zip_codes, axis=1)
    return df

# Calcular o volume do produto
df['product_volume'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']

# Adicionar a variável de frete (freight_value)
df['freight_value'] = df['freight_value']

df = extract_datetime_features(df)
df = calculate_zip_similarity(df)

# Preencher valores ausentes com a mediana das colunas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Variáveis preditoras e alvo, incluindo order_id e order_purchase_timestamp para uso futuro
X = df[['order_id', 'order_purchase_timestamp', 'delivery_time_model', 'purchase_weekday', 'purchase_month', 'purchase_hour', 'approval_delay', 'product_volume', 'product_weight_g', 'zip_code_similarity', 'freight_value', 'price']]
y = df['delivery_time']  # Usar delivery_time já existente

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline de pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['approval_delay', 'product_volume', 'product_weight_g', 'zip_code_similarity', 'freight_value', 'price']),
        ('cat', OneHotEncoder(), ['purchase_weekday', 'purchase_month', 'purchase_hour'])
    ])

# Pipeline completo com regressor RandomForestRegressor
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
])

# Treinar o modelo
model_pipeline.fit(X_train.drop(columns=['order_id', 'order_purchase_timestamp','delivery_time_model']), y_train)

# Fazer previsões nos dados de teste
y_pred = model_pipeline.predict(X_test.drop(columns=['order_id', 'order_purchase_timestamp', 'delivery_time_model']))

# Diagnóstico para verificar as previsões
print("Valores de previsão extremos:", y_pred[(y_pred < 0) | (y_pred > 365)]) 

# Limitar valores de previsão a um intervalo razoável, por exemplo, 0 a 365 dias (1 ano)
y_pred = np.clip(y_pred, 0, 365)

# Criar DataFrame com as previsões e os resultados reais
df_resultado = pd.DataFrame({
    'OrderID': X_test['order_id'].values,
    'Delivery Time Real': y_test,
    'Delivery Time Previsto (Antigo)': X_test['delivery_time_model'],
    'Delivery Time Previsto (Novo)': y_pred,
})

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'MSE: {mse}')
print(f'MAE: {mae}')

# Salvar o modelo para uso futuro
joblib.dump(model_pipeline, 'model_pipeline.pkl')

# Salvar resultado como CSV
df_resultado.to_csv('Data/resultado_previsoes.csv', index=False)

print("CSV de previsões salvo com sucesso!")


Valores de previsão extremos: []
MSE: 18.4067136434971
MAE: 3.1528348047677763
CSV de previsões salvo com sucesso!


In [17]:
df_resultado

,OrderID,Delivery Time Real,Delivery Time Previsto (Antigo),Delivery Time Previsto (Novo)
18527,4912377c02eb84d181d53e98e010124e,8.81,18.20,9.9720
49324,c25acd72d5c2eb2ec89c3c8d7cd7d1f2,4.26,11.50,7.6606
43893,ad4c4ee12613fc3dc5d506e4e6455498,16.61,41.01,16.7375
22937,5a5a4a1319679a5e560257abd2fa1273,12.83,25.20,12.8334
18282,48251a875839d15f52e7dcb61e93628a,10.34,32.35,10.8214
...,...,...,...,...
39984,9e5fd9b33580d845c54855ee2e3965b0,15.15,28.38,10.6027
63102,f8b730c3feb3136010e89c9acd18cdad,5.30,17.32,5.0609
12678,3273805a17a78c15811a6feb489662fd,9.06,27.33,9.4697
26392,68233f255c4f267082b2fc19f6449977,26.54,21.96,11.8614


In [11]:
# Avaliar o modelo que recebemos na AWS
mse = mean_squared_error(y_test, X_test['delivery_time_model'])
mae = mean_absolute_error(y_test, X_test['delivery_time_model'])
print(f'MSE: {mse}')
print(f'MAE: {mae}')

MSE: 215.1133719944513
MAE: 13.011396424167694


In [ ]:
# Código para imprimir a data específica que estamos prevendo que o pedido será entregue, mas como não precisamos disso no kaggle estou deixando de lado por enquanto

'PrevisaoEntrega': [X_test['order_purchase_timestamp'].iloc[i] + timedelta(days=pred) for i, pred in enumerate(y_pred)]


In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib

# Carregar os dados
df = pd.read_csv("Data/DataFramelimpa_sem_latlong.csv")

# Pré-processamento dos Dados
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# Função para extrair características de data/hora
def extract_datetime_features(df):
    df['purchase_weekday'] = df['order_purchase_timestamp'].dt.weekday
    df['purchase_month'] = df['order_purchase_timestamp'].dt.month
    df['purchase_hour'] = df['order_purchase_timestamp'].dt.hour
    df['approval_delay'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # em horas
    return df

# Função para calcular a similaridade dos prefixos de CEP
def calculate_zip_similarity(df):
    def compare_zip_codes(row):
        seller_zip = str(row['seller_zip_code_prefix'])
        customer_zip = str(row['customer_zip_code_prefix'])
        similarity = sum(1 for s, c in zip(seller_zip, customer_zip) if s == c)
        return similarity

    df['zip_code_similarity'] = df.apply(compare_zip_codes, axis=1)
    return df

# Calcular o volume do produto
df['product_volume'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']

df = extract_datetime_features(df)
df = calculate_zip_similarity(df)

# Preencher valores ausentes com a mediana das colunas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Variáveis preditoras e alvo, incluindo order_id e order_purchase_timestamp para uso futuro
X = df[['order_id', 'order_purchase_timestamp', 'delivery_time_model', 'purchase_weekday', 'purchase_month', 'purchase_hour', 'approval_delay', 'product_volume', 'product_weight_g', 'zip_code_similarity', 'freight_value', 'price']]
y = df['delivery_time']  # Usar delivery_time já existente

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline de pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['approval_delay', 'product_volume', 'product_weight_g', 'zip_code_similarity', 'freight_value', 'price']),
        ('cat', OneHotEncoder(), ['purchase_weekday', 'purchase_month', 'purchase_hour'])
    ])

# Pipeline completo com regressor RandomForestRegressor
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42, n_jobs=-1))
])

# Definir a grade de hiperparâmetros
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4],
    'regressor__bootstrap': [True, False]
}

# GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(estimator=model_pipeline, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')

# Treinar o modelo com GridSearch
grid_search.fit(X_train.drop(columns=['order_id', 'order_purchase_timestamp', 'delivery_time_model']), y_train)

# Melhor modelo encontrado
best_model = grid_search.best_estimator_

# Fazer previsões nos dados de teste
y_pred = best_model.predict(X_test.drop(columns=['order_id', 'order_purchase_timestamp', 'delivery_time_model']))

# Diagnóstico para verificar as previsões
print("Valores de previsão extremos:", y_pred[(y_pred < 0) | (y_pred > 365)]) 

# Limitar valores de previsão a um intervalo razoável, por exemplo, 0 a 365 dias (1 ano)
y_pred = np.clip(y_pred, 0, 365)

# Criar DataFrame com as previsões e os resultados reais
df_resultado = pd.DataFrame({
    'OrderID': X_test['order_id'].values,
    'Delivery Time Real': y_test,
    'Delivery Time Previsto (Antigo)': X_test['delivery_time_model'],
    'Delivery Time Previsto (Novo)': y_pred,
})

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'Melhores hiperparâmetros: {grid_search.best_params_}')

# Salvar o modelo para uso futuro
joblib.dump(best_model, 'best_model_pipeline.pkl')

# Salvar resultado como CSV
df_resultado.to_csv('Data/resultado_previsoes.csv', index=False)

print("CSV de previsões salvo com sucesso!")


In [18]:
# código que retora o csv do jeito que ele precisa ser submetido no kaggle

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
import joblib
from datetime import datetime, timedelta

# Função para extrair características de data/hora
def extract_datetime_features(df):
    df['purchase_weekday'] = df['order_purchase_timestamp'].dt.weekday
    df['purchase_month'] = df['order_purchase_timestamp'].dt.month
    df['purchase_hour'] = df['order_purchase_timestamp'].dt.hour
    df['approval_delay'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # em horas
    return df

# Função para calcular a similaridade dos prefixos de CEP
def calculate_zip_similarity(df):
    def compare_zip_codes(row):
        seller_zip = str(row['seller_zip_code_prefix'])
        customer_zip = str(row['customer_zip_code_prefix'])
        similarity = sum(1 for s, c in zip(seller_zip, customer_zip) if s == c)
        return similarity

    df['zip_code_similarity'] = df.apply(compare_zip_codes, axis=1)
    return df

# Função para pré-processamento dos dados
def preprocess_data(df):
    df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
    df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])
    df['product_volume'] = df['product_length_cm_x'] * df['product_height_cm_x'] * df['product_width_cm_x']
    df['product_weight_g'] = df['product_weight_g_x']
    df = extract_datetime_features(df)
    df = calculate_zip_similarity(df)
    df = df.fillna(0)
    return df

# Carregar o modelo treinado
model_pipeline = joblib.load('model_pipeline.pkl')

# Carregar os novos dados
df_new = pd.read_csv("Data/kaggle_enriquecido.csv")
df_new = preprocess_data(df_new)

# Variáveis preditoras
X_new = df_new[['order_id', 'order_purchase_timestamp', 'purchase_weekday', 'purchase_month', 'purchase_hour', 'approval_delay', 'product_volume', 'product_weight_g', 'zip_code_similarity', 'freight_value', 'price']]

# Fazer previsões nos novos dados
y_pred_new = model_pipeline.predict(X_new.drop(columns=['order_id', 'order_purchase_timestamp']))

# Criar DataFrame com as previsões
df_resultado_new = pd.DataFrame({
    'order_id': X_new['order_id'].values,
    'order_metric_cycle_time': y_pred_new
})

# Salvar resultado como CSV sem cabeçalhos
df_resultado_new.to_csv('Data/resultado_previsoes_kaggle2.csv', index=False)

print("CSV de previsões para o kaggle salvo com sucesso!")
print(df_resultado_new.head()) 


CSV de previsões para o kaggle salvo com sucesso!
                           order_id  order_metric_cycle_time
0  cee3292f46ede6ea1dfabfcb200fcf47                   9.5672
1  50dca53ca33b739bef09e7933e8b380e                  10.2296
2  8087ec71e393d4dc6fc48041fe63cd51                   5.8191
3  e6b6557ce111de79b31cc857f20ba212                  10.3058
4  0b09c5e4c2512f627190ac55a78c35a3                  14.2867


In [ ]:
df_resultado_new = pd.read_csv('Data/resultado_previsoes_kaggle.csv')

# Descrever a coluna 'Delivery Time Previsão'
delivery_time_stats = df_resultado_new['Delivery Time Previsão'].describe()

delivery_time_stats

count    28152.000000
mean        10.936476
std          3.431379
min          2.430900
25%          8.398900
50%         11.341250
75%         13.487275
max         22.471000
Name: Delivery Time Previsão, dtype: float64

In [ ]:
import pandas as pd

# Carregar a DataFrame principal e a DataFrame order_items
df_principal = pd.read_csv("Data/Dataframelimpa_sem_latlong.csv")
df_order_items = pd.read_csv("Data/order_items.csv")

# Corrigir zip codes
df_principal['seller_zip_code_prefix'] = df_principal['seller_zip_code_prefix'].astype(str).apply(lambda x: x.zfill(5) if len(x) == 4 else x)
df_principal['customer_zip_code_prefix'] = df_principal['customer_zip_code_prefix'].astype(str).apply(lambda x: x.zfill(5) if len(x) == 4 else x)

# Remover linhas duplicadas da DataFrame order_items com base na coluna order_id
df_order_items_clean = df_order_items.drop_duplicates(subset=['order_id'])

# Realizar o merge usando pandas
df_final = pd.merge(df_principal, df_order_items_clean, on='order_id', how='left')

# Verificar o tamanho da DataFrame final
print(f"Número de linhas na DataFrame final: {len(df_final)}")

df_final.to_csv("Data/DataFrame_final_com_juncao.csv", index=False)


Número de linhas na DataFrame final: 59352


In [ ]:
# código que rodou por 13 horas procurando hiper parametros e retornou um mse quase igual

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from concurrent.futures import ThreadPoolExecutor
import joblib

# Carregar os dados
df = pd.read_csv("Data/Dataframelimpa_sem_latlong.csv")

df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

# Função para extrair características de data/hora
def extract_datetime_features(df):
    df['purchase_weekday'] = df['order_purchase_timestamp'].dt.weekday
    df['purchase_month'] = df['order_purchase_timestamp'].dt.month
    df['purchase_hour'] = df['order_purchase_timestamp'].dt.hour
    df['approval_delay'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # em horas
    return df

df = extract_datetime_features(df)

# Variáveis preditoras e alvo
X = df[['purchase_weekday', 'purchase_month', 'purchase_hour', 'approval_delay']]
y = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600  # Prever em horas

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline de pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['approval_delay']),
        ('cat', OneHotEncoder(), ['purchase_weekday', 'purchase_month', 'purchase_hour'])
    ])

# Pipeline completo
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# GridSearchCV para encontrar os melhores hiperparâmetros
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4]
}

# Função para treinar um modelo com um conjunto de parâmetros
def train_model(params):
    model_pipeline.set_params(**params)
    model_pipeline.fit(X_train, y_train)
    return model_pipeline

# Usar ThreadPoolExecutor para paralelizar GridSearchCV
with ThreadPoolExecutor() as executor:
    grid_search = GridSearchCV(model_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# Fazer previsões
y_pred = best_model.predict(X_test)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse}')

# Salvar o modelo para uso futuro
joblib.dump(best_model, 'modelo_previsao_data_entrega.pkl')

# Exemplo de uso do modelo salvo em novos dados
modelo_carregado = joblib.load('modelo_previsao_data_entrega.pkl')


MSE: 18843.643010133725


In [ ]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'])

df_novo = extract_datetime_features(df)

X_novo = df_novo[['purchase_weekday', 'purchase_month', 'purchase_hour', 'approval_delay']]
previsoes = modelo_carregado.predict(X_novo)

df_novo['previsao_data_entrega_horas'] = previsoes
df_novo['previsao_data_entrega'] = df_novo['order_purchase_timestamp'] + pd.to_timedelta(df_novo['previsao_data_entrega_horas'], unit='h')

print(df_novo)